# Analyse des logs d’un système de dessin assisté par IA

Ce notebook présente l’analyse des données issues d’un prototype d’application de dessin
permettant de comparer une activité de création graphique **avec** et **sans assistance par intelligence artificielle (IA)**.

L’objectif général du projet est d’évaluer l’impact de différentes formes d’assistance IA
(suggestions automatiques, assistant contextuel, génération d’éléments graphiques)
sur la réalisation de tâches de dessin guidées.

## Contexte expérimental

Les participants utilisent une application de dessin pour reproduire **trois dessins successifs** :
1. un chat,
2. un château,
3. une voiture.

Pour chaque dessin :
- un exemple à reproduire est affiché dans une fenêtre secondaire ;
- l’utilisateur dessine librement dans l’éditeur principal ;
- lorsqu’il estime le dessin terminé, il clique sur un bouton *Done* ;
- il renseigne ensuite une **auto-évaluation subjective de ressemblance** via un slider.

Les trois tâches sont réalisées dans un ordre fixe (chat → château → voiture).

## Conditions expérimentales

Deux conditions expérimentales sont comparées :

- **Sans IA (H_ONLY)** :  
  l’utilisateur dispose uniquement des outils classiques de dessin
  (stylo, formes, sélection, suppression, etc.).

- **Avec IA (H_PLUS_IA)** :  
  l’utilisateur a accès à plusieurs fonctionnalités d’assistance :
  - un assistant contextuel,
  - des suggestions automatiques,
  - un module de génération d’éléments graphiques par IA.

Chaque participant est affecté à **une seule condition expérimentale**.

## Objectifs de l’analyse

L’analyse vise à déterminer si l’assistance par IA :

- permet de **réduire le temps de réalisation** des dessins ;
- modifie le **comportement de dessin** des utilisateurs
  (en particulier les actions de correction) ;
- est **utilisée différemment selon le type de tâche** proposée.

Pour cela, nous exploitons les logs événementiels produits par l’application
lors de l’exécution des sessions de dessin.

## Questions de recherche

Cette analyse s’articule autour des questions de recherche suivantes :

- **RQ1** : l’assistance par IA permet-elle de réduire le temps nécessaire
  pour réaliser une tâche de dessin ?

- **RQ2** : l’assistance par IA modifie-t-elle le comportement de dessin,
  notamment en réduisant le nombre de corrections effectuées ?

- **RQ3** : l’usage de l’IA varie-t-il en fonction de la tâche
  (chat, château, voiture) ?

# Hypothèses expérimentales

À partir des objectifs du projet et des questions de recherche formulées précédemment,
nous définissons un ensemble d’hypothèses expérimentales visant à évaluer l’impact
de l’assistance par intelligence artificielle (IA) sur la réalisation de tâches de dessin.

Chaque hypothèse est associée à des variables observables issues des logs
et sera testée à l’aide d’analyses statistiques appropriées dans les sections suivantes.

## H1 – Hypothèse descriptive (efficacité temporelle)

**H1** : le temps de réalisation d’un dessin est plus faible lorsque l’utilisateur
dispose d’une assistance par IA que lorsqu’il n’en dispose pas.

- **Variable indépendante (VI)** :  
  condition expérimentale (*Sans IA* vs *Avec IA*).

- **Variable dépendante (VD)** :  
  durée de réalisation d’un essai, mesurée en secondes.

- **Attente théorique** :  
  l’assistance par IA facilite certaines étapes de la production graphique,
  ce qui se traduit par une réduction du temps nécessaire pour finaliser un dessin.

## H2 – Hypothèse comportementale (corrections)

**H2** : le nombre de corrections effectuées lors de la réalisation d’un dessin
est plus faible lorsque l’utilisateur dispose d’une assistance par IA.

- **Variable indépendante (VI)** :  
  condition expérimentale (*Sans IA* vs *Avec IA*).

- **Variable dépendante (VD)** :  
  nombre d’actions de correction effectuées au cours d’un essai.

- **Attente théorique** :  
  l’assistance par IA permettrait de guider l’utilisateur vers des tracés plus
  adaptés dès les premières tentatives, réduisant ainsi le besoin de modifier
  ou d’annuler des actions précédentes.

## H3 – Hypothèse d’interaction (usage de l’IA selon la tâche)

**H3** : l’assistance par IA n’est pas utilisée de la même manière selon le type
de tâche de dessin proposé.

- **Variable indépendante (VI)** :  
  type de tâche (*chat*, *château*, *voiture*).

- **Variable dépendante (VD)** :  
  indicateurs d’usage de l’IA au cours d’un essai
  (par exemple : nombre d’éléments générés par IA ou utilisation effective de l’IA).

- **Attente théorique** :  
  certaines tâches, plus structurées ou composées d’éléments répétitifs,
  pourraient inciter davantage à recourir aux fonctionnalités d’assistance par IA
  que d’autres tâches plus "détaillées".

## Synthèse des hypothèses

Les hypothèses formulées couvrent trois dimensions complémentaires :

- **H1** porte sur l’efficacité globale de la tâche (temps de réalisation) ;
- **H2** porte sur le comportement de l’utilisateur lors du dessin
  (actions de correction) ;
- **H3** porte sur l’interaction entre l’utilisateur, l’IA et le type de tâche.

Les sections suivantes détaillent la manière dont ces hypothèses sont
opérationnalisées à partir des logs et testées statistiquement.

# Données et schéma des logs

Cette section décrit les données collectées lors de l’expérimentation,
leur structure générale, ainsi que les types d’événements enregistrés par l’application.

Les données analysées proviennent de fichiers de logs au format CSV,
dans lesquels chaque ligne correspond à un **événement horodaté**
survenu lors d’une session utilisateur.

In [1]:
import pandas as pd

# Chargement du fichier de logs principal
df = pd.read_csv("logs/events_all.csv")

# Conversion du timestamp si nécessaire
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")

# Tri chronologique par session
df = df.sort_values(["session_id", "timestamp"])

df.head()

,timestamp,session_id,condition,task_id,trial_index,event_type,tool,item_type,n_points,notes
929,2026-01-20 06:40:32.909178,05ea834f-1168-4844-b879-efb56470281e,NaN,NaN,NaN,stroke_color_change,NaN,NaN,NaN,NaN
930,2026-01-20 06:40:32.915704,05ea834f-1168-4844-b879-efb56470281e,NaN,NaN,NaN,fill_color_change,NaN,NaN,NaN,NaN
931,2026-01-20 06:40:36.697314,05ea834f-1168-4844-b879-efb56470281e,H_ONLY,NaN,NaN,test_start,NaN,NaN,NaN,condition=H_ONLY;3_tasks_fixed_order
932,2026-01-20 06:40:36.697949,05ea834f-1168-4844-b879-efb56470281e,H_ONLY,NaN,NaN,assistant_auto_toggle,NaN,NaN,NaN,False
933,2026-01-20 06:40:36.698159,05ea834f-1168-4844-b879-efb56470281e,H_ONLY,NaN,NaN,assistant_floating_toggle,NaN,NaN,NaN,False


## Volume et granularité des données

Les logs sont enregistrés à un niveau **événementiel** :
chaque interaction significative de l’utilisateur avec l’application
donne lieu à l’enregistrement d’une ligne dans le fichier de logs.

Ainsi :
- une session utilisateur contient plusieurs centaines d’événements ;
- chaque session est composée de plusieurs essais (*trials*) ;
- les analyses nécessitent une **agrégation** des événements
  afin d’obtenir des métriques interprétables au niveau de l’essai
  ou de la session.

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2404 entries, 929 to 159
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   timestamp    2404 non-null   datetime64[ns]
 1   session_id   2404 non-null   object        
 2   condition    2380 non-null   object        
 3   task_id      2344 non-null   object        
 4   trial_index  2344 non-null   float64       
 5   event_type   2404 non-null   object        
 6   tool         2032 non-null   object        
 7   item_type    678 non-null    object        
 8   n_points     18 non-null     float64       
 9   notes        734 non-null    object        
dtypes: datetime64[ns](1), float64(2), object(7)
memory usage: 206.6+ KB


## Sessions et essais

Chaque participant correspond à une **session** identifiée par un identifiant unique
(`session_id`).

Au sein d’une session :
- l’utilisateur réalise trois essais successifs (*trials*) ;
- chaque essai correspond à la reproduction d’un dessin spécifique
  (*chat*, *château* ou *voiture*) ;
- les essais sont délimités par des événements de début et de fin
  explicitement enregistrés dans les logs.

## Événements structurants

Certains types d’événements jouent un rôle structurant dans les logs :

- `trial_start` : début d’un essai ;
- `trial_end` : fin d’un essai (incluant la durée totale) ;
- `done_clicked` : validation explicite par l’utilisateur
  de la fin de son dessin.

Ces événements permettent de segmenter les logs événementiels
en essais distincts, qui constituent l’unité principale d’analyse
dans la suite du notebook.

## Colonnes principales des logs

Les colonnes suivantes sont particulièrement importantes pour l’analyse :

- `session_id` : identifiant de la session utilisateur ;
- `condition` : condition expérimentale (*Sans IA* / *Avec IA*) ;
- `task_id` : type de dessin à réaliser (chat, château, voiture) ;
- `trial_index` : index de l’essai dans la session ;
- `event_type` : type d’événement enregistré ;
- `timestamp` : horodatage de l’événement ;
- `notes` : informations complémentaires associées à l’événement.

Ces champs fournissent à la fois le **contexte expérimental**
et le **contenu comportemental** des interactions utilisateur.

## Typologie des événements comportementaux

Les événements enregistrés peuvent être regroupés en plusieurs catégories :

- **Événements de contrôle expérimental**  
  (ex. début et fin d’essai, validation de tâche).

- **Actions de dessin**  
  (tracés, création de formes, déplacements d’objets).

- **Actions de correction**  
  (suppression, annulation, modification d’éléments existants).

- **Événements liés à l’assistance par IA**  
  (génération d’éléments, suggestions automatiques, interactions avec l’assistant).

Cette typologie guidera la définition des métriques utilisées
pour tester les hypothèses formulées précédemment.

## Vers les prétraitements

Étant donné la nature événementielle et volumineuse des logs,
un ensemble de prétraitements est nécessaire afin de :

- vérifier la cohérence des sessions ;
- identifier les essais complets exploitables ;
- agréger les événements en métriques synthétiques
  au niveau de l’essai et de la session.

La section suivante détaille ces étapes de prétraitement
et les critères d’exclusion appliqués.

# Prétraitements et critères d’exclusion

Avant de tester les hypothèses formulées précédemment,
il est nécessaire de vérifier la cohérence des données collectées
et d’appliquer des prétraitements afin d’obtenir des unités d’analyse exploitables.

Cette section décrit :
- les contrôles de cohérence effectués sur les logs ;
- les critères d’exclusion appliqués, le cas échéant ;
- les décisions méthodologiques prises avant l’analyse statistique.

In [7]:
# Nombre total de sessions enregistrées
n_sessions_total = df["session_id"].nunique()

n_sessions_total

12

## Complétude des essais

Dans le protocole expérimental, chaque session valide doit contenir
trois essais complets, correspondant aux trois tâches de dessin
(chat, château et voiture).

Un essai est considéré comme complet s’il contient :
- un événement de début d’essai (`trial_start`) ;
- un événement de fin d’essai (`trial_end`).

Les sessions ne respectant pas cette structure sont considérées
comme incomplètes et ne sont pas retenues pour l’analyse.

In [8]:
# Comptage des événements trial_start et trial_end par session
trial_events = (
    df[df["event_type"].isin(["trial_start", "trial_end"])]
    .groupby(["session_id", "event_type"])
    .size()
    .unstack(fill_value=0)
)

trial_events.head()

event_type,trial_end,trial_start
session_id,,
05ea834f-1168-4844-b879-efb56470281e,3,3
0944ec51-8d5c-4d1f-b85f-f2dccee72a9d,3,3
174125af-8789-43c9-9c88-41f164004e5c,3,3
3a988a3d-fa10-4ebc-9d04-9b8471df4b89,3,3
3b5d2a63-8d25-46fe-af81-1c0b893bbee6,3,3


In [9]:
# Sessions avec exactement 3 débuts et 3 fins d'essais
complete_sessions = trial_events[
    (trial_events.get("trial_start", 0) == 3) &
    (trial_events.get("trial_end", 0) == 3)
].index

n_sessions_complete = len(complete_sessions)

n_sessions_complete

12

In [10]:
# Conservation uniquement des sessions complètes
df_clean = df[df["session_id"].isin(complete_sessions)].copy()

df_clean["session_id"].nunique()

12

## Vérification des conditions expérimentales

Chaque session est associée à une unique condition expérimentale
(*Sans IA* ou *Avec IA*).

Avant l’analyse, nous vérifions que :
- chaque session possède une condition renseignée ;
- la répartition des sessions entre conditions est identifiable.

In [11]:
# Répartition des sessions par condition
sessions_conditions = (
    df_clean[["session_id", "condition"]]
    .drop_duplicates()
    .value_counts("condition")
)

sessions_conditions

condition
H_ONLY       6
H_PLUS_IA    6
Name: count, dtype: int64

## Vérification des champs contextuels

Les analyses ultérieures reposent sur plusieurs champs contextuels
(`task_id`, `trial_index`, `condition`).

Nous vérifions que ces champs sont bien renseignés
pour les événements appartenant à des essais complets,
afin d’éviter toute ambiguïté lors de l’agrégation des données.

In [12]:
# Vérification des valeurs manquantes pour les champs critiques
critical_fields = ["session_id", "condition", "task_id", "trial_index", "event_type"]

df_clean[critical_fields].isna().sum()

session_id      0
condition      24
task_id        60
trial_index    60
event_type      0
dtype: int64

## Interprétation des valeurs manquantes dans les champs contextuels

Certaines colonnes contextuelles (`condition`, `task_id`, `trial_index`)
présentent des valeurs manquantes pour une partie des événements.

Ces valeurs manquantes correspondent à des événements **hors essai**
(par exemple : début ou fin de session, événements globaux de l’interface),
pour lesquels l’association à une tâche ou à un essai n’est pas pertinente.

Dans la suite de l’analyse :
- seuls les événements appartenant à un essai seront considérés ;
- ces événements sont identifiés par la présence conjointe
  d’un `task_id` et d’un `trial_index` renseignés.

Cette restriction permet de garantir que les métriques calculées
se rapportent exclusivement à des interactions liées à une tâche de dessin.

In [13]:
# Sélection des événements appartenant à un essai
df_trials_events = df_clean[
    df_clean["task_id"].notna() &
    df_clean["trial_index"].notna()
].copy()

df_trials_events[["task_id", "trial_index"]].isna().sum()

task_id        0
trial_index    0
dtype: int64

## Interprétation des valeurs manquantes de la condition expérimentale

La colonne `condition` présente des valeurs manquantes pour certains événements.

Cela s’explique par le fait que la condition expérimentale est une propriété
de la **session utilisateur**, et non de chaque événement pris individuellement.
Certains événements (initialisation, événements globaux de l’interface)
surviennent en dehors de la phase de test et ne nécessitent pas
d’association explicite à une condition.

Dans la suite de l’analyse :
- la condition est définie **une seule fois par session** ;
- elle est ensuite associée aux essais et aux métriques agrégées
  par jointure avec une table de référence session → condition.

Cette approche garantit une comparaison correcte entre conditions
dans un protocole expérimental de type *between-subjects*.

In [14]:
# Vérification des conditions par session (sur les sessions complètes)
conditions_per_session = (
    df_clean[["session_id", "condition"]]
    .dropna(subset=["condition"])
    .drop_duplicates()
    .groupby("session_id")
    .size()
)

conditions_per_session.value_counts()

1    12
Name: count, dtype: int64

In [15]:
# Table de référence : une condition par session
session_condition = (
    df_clean[["session_id", "condition"]]
    .dropna()
    .drop_duplicates()
)

session_condition.head()

,session_id,condition
931,05ea834f-1168-4844-b879-efb56470281e,H_ONLY
433,0944ec51-8d5c-4d1f-b85f-f2dccee72a9d,H_PLUS_IA
162,174125af-8789-43c9-9c88-41f164004e5c,H_ONLY
804,3a988a3d-fa10-4ebc-9d04-9b8471df4b89,H_PLUS_IA
2013,3b5d2a63-8d25-46fe-af81-1c0b893bbee6,H_ONLY


## Synthèse des prétraitements

À l’issue de ces étapes de prétraitement :

- seules les sessions contenant trois essais complets sont conservées ;
- chaque session retenue est associée à une condition expérimentale unique ;
- les événements hors essai sont identifiés et exclus
  des analyses comportementales ;
- les événements analysés sont tous associés à une tâche et à un essai ;
- la condition expérimentale est portée au niveau de la session
  et sera associée aux essais lors de l’agrégation.

Le dataset obtenu constitue une base cohérente et méthodologiquement
valide pour la définition des métriques et les analyses statistiques.

# Définition opérationnelle des métriques

Cette section définit les métriques utilisées pour rendre opérationnelles les variables dépendantes
associées aux hypothèses H1–H3.

Les logs étant enregistrés au niveau événementiel, les métriques sont calculées par agrégation
au niveau de l’**essai** (*trial*), défini par la paire (`session_id`, `trial_index`) et associé à une tâche (`task_id`).

Les métriques définies ci-dessous sont ensuite utilisées pour :
- construire une table `trials` (1 ligne = 1 essai) ;
- tester statistiquement les hypothèses expérimentales.

## Choix opérationnels : quels événements comptent pour quelles métriques ?

Afin de calculer des indicateurs comparables entre participants, nous définissons
des ensembles d’événements associés à des catégories comportementales.

Les définitions suivantes sont **opérationnelles** : elles s’appuient sur les types d’événements
enregistrés dans les logs, et peuvent être ajustées si le schéma de logs évolue.

Dans la suite :
- une **action** correspond à un événement reflétant une interaction de production/modification ;
- une **correction** correspond à un événement visant à modifier, annuler ou supprimer un contenu existant ;
- un **usage IA** correspond à un événement impliquant explicitement une fonctionnalité d’assistance.

In [16]:
# Inventaire rapide des event_type présents (utile pour ajuster les listes)
event_types = sorted(df_trials_events["event_type"].dropna().unique().tolist())
event_types[:50], len(event_types)

(['ai_output',
  'assistant_auto_toggle',
  'assistant_suppress',
  'done_clicked',
  'ellipse_end',
  'ellipse_start',
  'erase',
  'fill_color_change',
  'gen_add',
  'invoke_help',
  'item_moved',
  'line_end',
  'line_start',
  'pen_end',
  'pen_start',
  'rect_end',
  'rect_start',
  'select_press',
  'self_eval',
  'stroke_color_change',
  'test_end',
  'tool_change',
  'trial_end',
  'trial_start',
  'triangle_end',
  'triangle_start',
  'user_action'],
 27)

In [17]:
# --- Actions (production / manipulation) ---
# On inclut des événements typiques de fin d'action, car ils sont plus stables que les événements "move" continus.
ACTION_EVENTS = {
    "pen_end", "shape_end", "item_created", "item_moved", "item_resized",
    "item_rotated", "paste", "duplicate"
}

# --- Corrections ---
CORRECTION_EVENTS = {
    "erase", "delete", "cut", "undo", "redo", "clear"
}

# --- IA (assistance explicite) ---
# Dans ton projet, "gen_add" est un marqueur fort de génération.
IA_EVENTS = {
    "gen_add", "assistant_accept", "assistant_reject", "autosuggest_shown",
    "autosuggest_accept", "autosuggest_reject"
}

# Filtrage : on ne garde que ceux qui existent vraiment dans le dataset
ACTION_EVENTS = {e for e in ACTION_EVENTS if e in event_types}
CORRECTION_EVENTS = {e for e in CORRECTION_EVENTS if e in event_types}
IA_EVENTS = {e for e in IA_EVENTS if e in event_types}

ACTION_EVENTS, CORRECTION_EVENTS, IA_EVENTS

({'item_moved', 'pen_end'}, {'erase'}, {'gen_add'})